In [22]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [23]:
AS= pd.read_csv('../data/Delta_Airlines.csv',skiprows=7)


In [24]:
df= pd.DataFrame(AS)
print(len(df))

139343


In [25]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
1039,DL,01/03/2018,1057.0,N859DN,LAS,07:00,00:00
1043,DL,01/03/2018,1162.0,N709TW,LAX,06:00,00:00
1048,DL,01/03/2018,1269.0,N966AT,TPA,22:17,00:00
1386,DL,01/04/2018,4.0,N713TW,SLC,17:59,00:00
1387,DL,01/04/2018,40.0,N1200K,LAX,17:52,00:00
...,...,...,...,...,...,...,...
137008,DL,12/25/2015,234.0,N710TW,LAX,19:54,00:00
137025,DL,12/25/2015,676.0,N699DL,STT,17:50,00:00
137047,DL,12/25/2015,1972.0,N845DN,SEA,15:28,00:00
137064,DL,12/25/2015,2604.0,N707TW,SFO,21:41,00:00


In [26]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,DL,01/01/2015,16.0,N199DN,LAX,17:36,17:03
1,DL,01/01/2015,246.0,N391DA,SAN,14:59,14:28
2,DL,01/01/2015,326.0,N3750D,SJU,10:54,10:57
3,DL,01/01/2015,332.0,N686DA,SJU,20:38,20:42
4,DL,01/01/2015,348.0,N373DA,SJU,23:15,23:12


In [27]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [28]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,DL,01/01/2015,16.0,N199DN,LAX,17:36,17:03
1,DL,01/01/2015,246.0,N391DA,SAN,14:59,14:28
2,DL,01/01/2015,326.0,N3750D,SJU,10:54,10:57
3,DL,01/01/2015,332.0,N686DA,SJU,20:38,20:42
4,DL,01/01/2015,348.0,N373DA,SJU,23:15,23:12


In [29]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,DL,2015-01-01,16.0,N199DN,LAX,17:36,17:03
1,DL,2015-01-01,246.0,N391DA,SAN,14:59,14:28
2,DL,2015-01-01,326.0,N3750D,SJU,10:54,10:57
3,DL,2015-01-01,332.0,N686DA,SJU,20:38,20:42
4,DL,2015-01-01,348.0,N373DA,SJU,23:15,23:12
...,...,...,...,...,...,...,...
139337,DL,2018-12-31,2805.0,N712TW,SLC,17:55,18:03
139338,DL,2018-12-31,2869.0,N988AT,TPA,17:58,17:42
139339,DL,2018-12-31,2896.0,N3755D,DEN,16:05,16:20
139340,DL,2018-12-31,2916.0,N867DN,PDX,14:27,14:27


In [30]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,DL,2015-01-01,16.0,N199DN,LAX,17:36,17:03,2015-01-01
1,DL,2015-01-01,246.0,N391DA,SAN,14:59,14:28,2015-01-01
2,DL,2015-01-01,326.0,N3750D,SJU,10:54,10:57,2015-01-01
3,DL,2015-01-01,332.0,N686DA,SJU,20:38,20:42,2015-01-01
4,DL,2015-01-01,348.0,N373DA,SJU,23:15,23:12,2015-01-01
...,...,...,...,...,...,...,...,...
139337,DL,2018-12-31,2805.0,N712TW,SLC,17:55,18:03,2018-12-31
139338,DL,2018-12-31,2869.0,N988AT,TPA,17:58,17:42,2018-12-31
139339,DL,2018-12-31,2896.0,N3755D,DEN,16:05,16:20,2018-12-31
139340,DL,2018-12-31,2916.0,N867DN,PDX,14:27,14:27,2018-12-31


In [31]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
1213,DL,2015-01-04,2262.0,N194DN,LAX,22:23,24:00,2015-01-04
2597,DL,2019-01-07,2496.0,N350NA,MSP,00:27,24:00,2019-01-07
4135,DL,2019-01-11,2826.0,N541US,ATL,23:59,24:00,2019-01-11
4451,DL,2018-01-12,2601.0,N344NW,MCO,22:34,24:00,2018-01-12
4692,DL,2017-01-13,523.0,N371DA,LAS,23:55,24:00,2017-01-13
...,...,...,...,...,...,...,...,...
135235,DL,2017-12-19,2214.0,N361NB,MSP,00:16,24:00,2017-12-19
135950,DL,2018-12-21,1076.0,N377NW,AUS,22:33,24:00,2018-12-21
136576,DL,2017-12-23,759.0,N191DN,LAX,00:17,24:00,2017-12-23
138431,DL,2015-12-29,2501.0,N396DA,LAS,00:09,24:00,2015-12-29


In [32]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [33]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
1039,DL,2018-01-03,1057.0,N859DN,LAS,07:00,00:00,2018-01-03
1043,DL,2018-01-03,1162.0,N709TW,LAX,06:00,00:00,2018-01-03
1048,DL,2018-01-03,1269.0,N966AT,TPA,22:17,00:00,2018-01-03
1213,DL,2015-01-04,2262.0,N194DN,LAX,22:23,00:00,2015-01-04
1386,DL,2018-01-04,4.0,N713TW,SLC,17:59,00:00,2018-01-04
...,...,...,...,...,...,...,...,...
137047,DL,2015-12-25,1972.0,N845DN,SEA,15:28,00:00,2015-12-25
137064,DL,2015-12-25,2604.0,N707TW,SFO,21:41,00:00,2015-12-25
137326,DL,2015-12-26,2350.0,N707TW,ATL,10:38,00:00,2015-12-26
138431,DL,2015-12-29,2501.0,N396DA,LAS,00:09,00:00,2015-12-29


In [34]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [35]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [36]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [37]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [38]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,DL,2015-01-01,16.0,N199DN,LAX,17:36,17:03,2015-01-01,2015-01-01 17:03,2015-01-01 17:36
1,DL,2015-01-01,246.0,N391DA,SAN,14:59,14:28,2015-01-01,2015-01-01 14:28,2015-01-01 14:59
2,DL,2015-01-01,326.0,N3750D,SJU,10:54,10:57,2015-01-01,2015-01-01 10:57,2015-01-01 10:54
3,DL,2015-01-01,332.0,N686DA,SJU,20:38,20:42,2015-01-01,2015-01-01 20:42,2015-01-01 20:38
4,DL,2015-01-01,348.0,N373DA,SJU,23:15,23:12,2015-01-01,2015-01-01 23:12,2015-01-01 23:15
...,...,...,...,...,...,...,...,...,...,...
139337,DL,2018-12-31,2805.0,N712TW,SLC,17:55,18:03,2018-12-31,2018-12-31 18:03,2018-12-31 17:55
139338,DL,2018-12-31,2869.0,N988AT,TPA,17:58,17:42,2018-12-31,2018-12-31 17:42,2018-12-31 17:58
139339,DL,2018-12-31,2896.0,N3755D,DEN,16:05,16:20,2018-12-31,2018-12-31 16:20,2018-12-31 16:05
139340,DL,2018-12-31,2916.0,N867DN,PDX,14:27,14:27,2018-12-31,2018-12-31 14:27,2018-12-31 14:27


In [39]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,DL,2015-01-01,16.0,N199DN,LAX,17:36,17:03,2015-01-01,2015-01-01 17:03:00,2015-01-01 17:36:00
1,DL,2015-01-01,246.0,N391DA,SAN,14:59,14:28,2015-01-01,2015-01-01 14:28:00,2015-01-01 14:59:00
2,DL,2015-01-01,326.0,N3750D,SJU,10:54,10:57,2015-01-01,2015-01-01 10:57:00,2015-01-01 10:54:00
3,DL,2015-01-01,332.0,N686DA,SJU,20:38,20:42,2015-01-01,2015-01-01 20:42:00,2015-01-01 20:38:00
4,DL,2015-01-01,348.0,N373DA,SJU,23:15,23:12,2015-01-01,2015-01-01 23:12:00,2015-01-01 23:15:00
...,...,...,...,...,...,...,...,...,...,...
139337,DL,2018-12-31,2805.0,N712TW,SLC,17:55,18:03,2018-12-31,2018-12-31 18:03:00,2018-12-31 17:55:00
139338,DL,2018-12-31,2869.0,N988AT,TPA,17:58,17:42,2018-12-31,2018-12-31 17:42:00,2018-12-31 17:58:00
139339,DL,2018-12-31,2896.0,N3755D,DEN,16:05,16:20,2018-12-31,2018-12-31 16:20:00,2018-12-31 16:05:00
139340,DL,2018-12-31,2916.0,N867DN,PDX,14:27,14:27,2018-12-31,2018-12-31 14:27:00,2018-12-31 14:27:00


In [40]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,DL,2015-01-01,16.0,N199DN,LAX,17:36,17:03,2015-01-01,2015-01-01 17:03:00,2015-01-01 17:36:00,33.0
1,DL,2015-01-01,246.0,N391DA,SAN,14:59,14:28,2015-01-01,2015-01-01 14:28:00,2015-01-01 14:59:00,31.0
2,DL,2015-01-01,326.0,N3750D,SJU,10:54,10:57,2015-01-01,2015-01-01 10:57:00,2015-01-01 10:54:00,-3.0
3,DL,2015-01-01,332.0,N686DA,SJU,20:38,20:42,2015-01-01,2015-01-01 20:42:00,2015-01-01 20:38:00,-4.0
4,DL,2015-01-01,348.0,N373DA,SJU,23:15,23:12,2015-01-01,2015-01-01 23:12:00,2015-01-01 23:15:00,3.0
...,...,...,...,...,...,...,...,...,...,...,...
139337,DL,2018-12-31,2805.0,N712TW,SLC,17:55,18:03,2018-12-31,2018-12-31 18:03:00,2018-12-31 17:55:00,-8.0
139338,DL,2018-12-31,2869.0,N988AT,TPA,17:58,17:42,2018-12-31,2018-12-31 17:42:00,2018-12-31 17:58:00,16.0
139339,DL,2018-12-31,2896.0,N3755D,DEN,16:05,16:20,2018-12-31,2018-12-31 16:20:00,2018-12-31 16:05:00,-15.0
139340,DL,2018-12-31,2916.0,N867DN,PDX,14:27,14:27,2018-12-31,2018-12-31 14:27:00,2018-12-31 14:27:00,0.0


In [41]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 21743
Flights that arrived early in 2019: 14970
Flights that arrived more than 20 mins early in 2019: 5619


In [42]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 30025
Flights that arrived early in 2018: 20699
Flights that arrived more than 20 mins early in 2018: 8060


In [43]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 29593
Flights that arrived early in 2017: 19713
Flights that arrived more than 20 mins early in 2017: 8470


In [44]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 30256
Flights that arrived early in 2016: 21153
Flights that arrived more than 20 mins early in 2016: 7438


In [45]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 27725
Flights that arrived early in 2015: 17731
Flights that arrived more than 20 mins early in 2015: 5355


In [46]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/Delta_Airlines.csv',index=False) 